# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
import torch as th

In [2]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [3]:
y = x + x

In [4]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [5]:
import syft as sy

In [6]:
hook = sy.TorchHook(th)

In [7]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [8]:
bob = sy.VirtualWorker(hook, id="bob") # simulates the interface that we might have to another machine

In [9]:
bob._objects

{}

In [10]:
x = th.tensor([1,2,3,4,5])

In [11]:
x = x.send(bob)

In [12]:
bob._objects

{81561658625: tensor([1, 2, 3, 4, 5])}

In [13]:
x.location # Would take the form of an IP address or another device identifier in the case of a real worker

<VirtualWorker id:bob #objects:1>

In [14]:
x.location == bob

True

In [15]:
x.id_at_location

81561658625

In [16]:
x.id

57883892779

In [17]:
# Whenever we execute a command to send or get info from bob, we are sending data between local worker and bob
x.owner

<VirtualWorker id:me #objects:0>

In [18]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [19]:
x

(Wrapper)>[PointerTensor | me:57883892779 -> bob:81561658625]

In [20]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [21]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [22]:
# try this project here!

In [23]:
x = th.tensor([1,2,3,4,5])

In [24]:
alice = sy.VirtualWorker(hook, id='alice')

In [25]:
x_ptr = x.send(bob, alice)

In [26]:
x_ptr.child

MultiPointerTensor>{'bob': [PointerTensor | me:7734451295 -> bob:5428449022], 'alice': [PointerTensor | me:9293555112 -> alice:5428449022]}

In [27]:
x_ptr.get()

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [28]:
x = th.tensor([1,2,3,4,5])
x_ptr = x.send(bob, alice)
x_ptr.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

# Lesson: Introducing Remote Arithmetic

In [29]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [30]:
x

(Wrapper)>[PointerTensor | me:57172687590 -> bob:7229508166]

In [31]:
y

(Wrapper)>[PointerTensor | me:17396996304 -> bob:7544690015]

In [32]:
z = x + y
z

(Wrapper)>[PointerTensor | me:1198246482 -> bob:21111403133]

In [33]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [34]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:81124225196 -> bob:86653196355]

In [35]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [36]:
# requries_grad -> requires gradient descent
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [37]:
z = (x + y).sum()

In [38]:
z.backward()

(Wrapper)>[PointerTensor | me:81773785479 -> bob:22717443507]

In [39]:
x = x.get()
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [40]:
x.grad

tensor([1., 1., 1., 1., 1.])

In [41]:
y = y.get()
y

tensor([1., 1., 1., 1., 1.], requires_grad=True)

In [42]:
y.grad

tensor([1., 1., 1., 1., 1.])

In [43]:
# Note that the reason that the gradients of x and y have been set to 1 after backpropagation
# 
z.get()

tensor(20., requires_grad=True)

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [44]:
# try this project here!

In [45]:
# Learn the simple relationship that whenever column two of the input data is 1,
# The output label is 1.
input_data = th.tensor([[1., 1], [0, 1], [1, 0], [0, 0]], requires_grad=True).send(bob)
labels = th.tensor([[1.], [1], [0], [0]], requires_grad=True).send(bob)

In [46]:
# Weights to perform backprop on
weights = th.tensor([[0.], [0.]], requires_grad=True).send(bob)

In [47]:
# TRAIN
epochs = 10
for i in range(epochs):
    
    # Loss Calcualtion
    prediction = input_data.mm(weights) # matmul
    loss = ((prediction - labels)**2).sum()
    loss.backward() # perform backprop on weights
        
    # Weights Adjustment
    weights.data.sub_(weights.grad * 0.1) # modify the weights based on result of backprop
    weights.grad *= 0 # reset the gradients
    
    # Print Result
    loss_data = loss.get().data
    print('Training iteration', i, 'Loss:', loss_data)
    
print('\nFinal Loss', loss_data)

Training iteration 0 Loss: tensor(2.)
Training iteration 1 Loss: tensor(0.5600)
Training iteration 2 Loss: tensor(0.2432)
Training iteration 3 Loss: tensor(0.1372)
Training iteration 4 Loss: tensor(0.0849)
Training iteration 5 Loss: tensor(0.0538)
Training iteration 6 Loss: tensor(0.0344)
Training iteration 7 Loss: tensor(0.0220)
Training iteration 8 Loss: tensor(0.0141)
Training iteration 9 Loss: tensor(0.0090)

Final Loss tensor(0.0090)


# Lesson: Garbage Collection and Common Errors


In [48]:
# Clear objects 
bob = bob.clear_objects()

In [49]:
bob._objects

{}

In [50]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [51]:
bob._objects

{51141902723: tensor([1, 2, 3, 4, 5])}

In [52]:
# In this case, x is a pointer to the tensor on bob's machine, if we delete the tensor, 
# we ALSO delete the data on Bob's machine. Turning garbage collection off disables this feature
x.child.garbage_collect_data

True

In [53]:
del x

In [54]:
bob._objects

{}

In [55]:
# Section demonstrating a gotcha when working in Jupter notebooks in which bob._objects
# doesn't get deleted even though it should, because jupyter keeps an extra reference to 
# bob in memeory even after x is deleted. This problem does not arise in normal python
# scripts

In [56]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [57]:
bob._objects

{55086867091: tensor([1, 2, 3, 4, 5])}

In [58]:
x = "asdf"

In [59]:
bob._objects

{}

In [60]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [61]:
x

(Wrapper)>[PointerTensor | me:9970569664 -> bob:80421129920]

In [62]:
bob._objects

{80421129920: tensor([1, 2, 3, 4, 5])}

In [63]:
x = "asdf"

In [64]:
bob._objects

{80421129920: tensor([1, 2, 3, 4, 5])}

In [65]:
del x

In [66]:
bob._objects # Bob's data is still 'believed' to exist by Jupyter, even though it doesn't

{80421129920: tensor([1, 2, 3, 4, 5])}

In [67]:
bob = bob.clear_objects()
bob._objects

{}

In [68]:
# If you assign bob 1000 tensors, but only save a reference to one, all of these thousand
# tensors is deleted from bob's machine, execpt the last tensor sent to him

# The purpose of this feature is to ensure that we won't generate a bunch of unreachable data
# on a worker machine and just clog up that machine with useless data
for i in range(1000):
    x = th.tensor([i,i,i,i,i]).send(bob)

In [69]:
bob._objects

{70658462206: tensor([999, 999, 999, 999, 999])}

In [70]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [71]:
# Demonstrating an error that occurs when we try to add a pointer to a tensor to a 
# standard torch tensor
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually located on another machine (is a PointerTensor). Call .get() on a the PointerTensor or .send({tensor_b.location.id}) on the other tensor.
Tensor A: {tensor_a}
Tensor B: {tensor_b}

In [ ]:
# Another common error in which you try to send tensors on two different machines
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [ ]:
z = x + y

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [ ]:
from torch import nn, optim

In [ ]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [ ]:
# A Toy Model
model = nn.Linear(2,1)

In [ ]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [ ]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum() # MSE

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

In [ ]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [ ]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [ ]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [ ]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

        print(loss.get())

In [ ]:
train()

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [ ]:
bob.clear_objects()
alice.clear_objects()

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
x = x.send(alice)

In [72]:
bob._objects

{70101567696: tensor([1, 2, 3, 4, 5])}

In [73]:
alice._objects

{}

In [74]:
y = x + x

In [75]:
y

(Wrapper)>[PointerTensor | me:10075344387 -> bob:45122732483]

In [76]:
bob._objects

{70101567696: tensor([1, 2, 3, 4, 5]),
 45122732483: tensor([ 2,  4,  6,  8, 10])}

In [77]:
alice._objects

{}

In [78]:
jon = sy.VirtualWorker(hook, id="jon")

In [79]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [80]:
bob._objects

{62313017188: tensor([1, 2, 3, 4, 5])}

In [81]:
alice._objects

{78983964185: (Wrapper)>[PointerTensor | alice:78983964185 -> bob:62313017188]}

In [82]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:78983964185 -> bob:62313017188]

In [83]:
bob._objects

{62313017188: tensor([1, 2, 3, 4, 5])}

In [84]:
alice._objects # Alice no longer has the data because she sent the pointer to us

{}

In [85]:
x = x.get() # Now, we get the DATA back from bob
x

tensor([1, 2, 3, 4, 5])

In [86]:
bob._objects

{}

In [87]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [88]:
bob._objects

{22764275835: tensor([1, 2, 3, 4, 5])}

In [89]:
alice._objects

{21950426558: (Wrapper)>[PointerTensor | alice:21950426558 -> bob:22764275835]}

In [90]:
del x

In [91]:
bob._objects

{}

In [92]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [93]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [94]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [95]:
bob._objects

{3567653335: tensor([1, 2, 3, 4, 5])}

In [96]:
alice._objects

{}

In [97]:
x.move(alice)

(Wrapper)>[PointerTensor | me:94248119000 -> alice:3567653335]

In [98]:
bob._objects

{}

In [99]:
alice._objects

{3567653335: tensor([1, 2, 3, 4, 5])}

In [100]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [101]:
bob._objects

{85882721973: tensor([1, 2, 3, 4, 5])}

In [102]:
alice._objects

{3567653335: tensor([1, 2, 3, 4, 5]),
 97472506871: (Wrapper)>[PointerTensor | alice:97472506871 -> bob:85882721973]}

In [103]:
x.remote_get()

(Wrapper)>[PointerTensor | me:41695118225 -> alice:97472506871]

In [104]:
bob._objects

{}

In [105]:
alice._objects

{3567653335: tensor([1, 2, 3, 4, 5]), 97472506871: tensor([1, 2, 3, 4, 5])}

In [106]:
x.move(bob)

(Wrapper)>[PointerTensor | me:41695118225 -> bob:97472506871]

In [107]:
x

(Wrapper)>[PointerTensor | me:41695118225 -> bob:97472506871]

In [108]:
bob._objects

{97472506871: tensor([1, 2, 3, 4, 5])}

In [109]:
alice._objects

{3567653335: tensor([1, 2, 3, 4, 5])}

# Section Project:

For the final project for this section, you're going to train on the MNIST dataset using federated learning However the gradient should not come up to central server in raw form

In [110]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as func
from torchvision import datasets, transforms

UnsupportedNodeError: import statements aren't supported:
  File "/root/miniconda3/lib/python3.8/site-packages/syft/generic/frameworks/hook/hook.py", line 2471
    .. include:: cuda_deterministic_backward.rst
    """
    from .modules.utils import _ntuple
    ~~~~ <--- HERE

    def _check_size_scale_factor(dim):
'_onnx_heatmaps_to_keypoints' is being compiled since it was called from '_onnx_heatmaps_to_keypoints_loop'
  File "/root/miniconda3/lib/python3.8/site-packages/torchvision/models/detection/roi_heads.py", line 218

    for i in range(int(rois.size(0))):
        xy_preds_i, end_scores_i = _onnx_heatmaps_to_keypoints(maps, maps[i],
        ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~...  <--- HERE
                                                               widths_ceil[i], heights_ceil[i],
                                                               widths[i], heights[i],


In [ ]:
bob.clear_objects()
alice.clear_objects()

### Set arguments up for ML Problem

In [ ]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = epochs
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

### Load Data

In [ ]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

### Specify CNN architecture

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = func.relu(self.conv1(x))
        x = func.max_pool2d(x, 2, 2)
        x = func.relu(self.conv2(x))
        x = func.max_pool2d(x, 2, 2)
        x = func.view(-1, 4*4*50)
        x = func.relu(self.fc1(x))
        x = self.fc2(x)
        return func.log_softmax(x, dim=1)

In [270]:
# Train function

In [271]:
def train(args, model, device, federated_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                100. * batch_idx / len(federated_train_loader), loss.item()))

In [272]:
# Test function

In [273]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [274]:
%%time
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

NameError: name 'Net' is not defined